In [ ]:
%matplotlib inline
from matplotlib import style
style.use('fivethirtyeight')
import matplotlib.pyplot as plt

import matplotlib.dates as mdates

In [ ]:
import numpy as np
import pandas as pd
import datetime as dt
from pprint import pprint


# Reflect Tables into SQLAlchemy ORM

In [ ]:
# Python SQL toolkit and Object Relational Mapper
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func

In [ ]:
# create engine to hawaii.sqlite
engine = create_engine("sqlite:///Resources/hawaii.sqlite")

In [ ]:
# reflect an existing database into a new model
Base = automap_base()
Base.prepare(engine, reflect=True)
keys = Base.classes.keys()
# reflect the tables
print(f'keys =')
print(*keys, sep = ', ')

In [ ]:
Measurement = Base.classes.measurement
Station = Base.classes.station

# Exploratory Precipitation Analysis

In [ ]:
# Find the most recent date in the data set.
session = Session(engine)
measure_first_row = session.query(Measurement).first()
measure_first_row.__dict__

In [ ]:
lastdate = session.query(func.max(Measurement.date)).\
             scalar()

dt_lastdate = dt.datetime.strptime(lastdate,"%Y-%m-%d").date()
dt_startdate = dt_lastdate - dt.timedelta(days=365)
startdate = dt_startdate.strftime("%Y-%m-%d")

query = session.query(Measurement).\
    filter(Measurement.date.between(startdate, lastdate)).\
    all()

session.close()

In [ ]:
dates = []
precip = []
for row in query:
    measurement = row
    dates.append(measurement.date)
    precip.append(measurement.prcp)

In [ ]:
measurement = pd.DataFrame({'date': dates,
                           'precipitation':precip})
measurement.dropna(inplace = True)
measurement

In [ ]:
measurement.set_index('date', inplace = True)
measurement.sort_index(inplace = True)
measurement

In [ ]:
plt.style.use('fivethirtyeight')

date = measurement.index
MDate = mdates.datestr2num(date)
precipitation = measurement['precipitation']

months_fmt = mdates.DateFormatter('%M')

fig, ax = plt.subplots()

ax.plot(MDate, precipitation, label = 'precipitation')

ax.xaxis.set_major_locator(mdates.MonthLocator())
ax.xaxis.set_major_formatter(mdates.DateFormatter("%Y-%m"))
fig.autofmt_xdate()

datemin = np.datetime64(date[0])
datemax = np.datetime64(date[-1])
precipmin = precipitation.min()
precipmax = precipitation.max() + .25
ax.set_xlim(datemin, datemax)
ax.set_ylim(precipmin, precipmax)

ax.grid(True)
ax.legend(loc=9)

ax.set_title("Precipitation Amounts (8/23/16 - 8/23/17)")
ax.set_ylabel("Precipitation (in)")
ax.set_xlabel("Dates")

plt.tight_layout

plt.savefig('results/precipitation.png')

plt.show

In [ ]:
precipitation.describe()

# Exploratory Station Analysis

In [ ]:
session = Session(engine)
station.__table__

In [ ]:
Measurement.__table__

In [ ]:
station_count = session.query(station).\
                distinct().\
                count()

print(f'there are {station_count} stations.')

In [ ]:
by_station_count = session.query(Measurement,
                                Measurement.station,
                                Station.name,
                                func.count(Measurement.station).label('station_count')).\
join(station, Measurement.station == Station.station).\
group_by(Measurement.station).\
order_by(func.count(Measurement.station).desc()).\
all()

for row in by_station_count:
    print (f'{row.station_count} observations were made at {row.name}')

In [ ]:
top_station_count = session.query(Measurement,
                                 Measurement.station,
                                 Station.name,
                                 func.count(Measurement.station).label("station_count")).\
join(Station, Measurement.station == Station.station).\
group_by(Measurement.station).\
order_by(func.count(Measurement.station).desc()).\
first()

print(f'{top_station_count.name} had the most observations with {top_station_count.station_count} observations.')

In [ ]:
top_station = session.query(Measurement.station).\
group_by(Measurement.station).\
order_by(func.count(Measurement.station).desc()).\
first()

t_station = top_station.station

In [ ]:
top_station_stats = session.query(Measurement,
                                 func.max(Measurement.tobs).label('max_temp'),
                                 func.min(Measurement.tobs).label('min_temp'),
                                 func.avg(Measurement.tobs).label('avg_temp'),
                                 Station.name.label('sta_name')).\
join(Station, Measurement.station == Station.station).\
filter(Measurement.station == t_station).\
first()

print(f'{top_station_stats.sta_name} had a maximum temperature of {top_station_stats.max_temp}')
print(f'{top_station_stats.sta_name} had a minimum temperature of {top_station_stats.min_temp}')
print(f'{top_station_stats.sta_name} had an average temperature of {top_station_stats.avg_temp}')
                                

In [ ]:
last12_top_sta_count = session.query(Measurement,
                                    Measurement.station,
                                    Station.name,
                                    func.count(Measurement.station).label('station_count')).\
join(Station, Measurement.station == Station.station).\
filter(Measurement.date.between(startdate,lastdate)).\
group_by(Measurement.station).\
order_by(func.count(Measurement.station).desc()).\
first()

print(f'{last12_top_sta_count.name} had the most observations in the last 12 months with {last12_top_sta_count.station_count} observations')

In [ ]:
subquery = session.query(Station.station).filter(Station.name.like('%WAIKIKI%')).subquery()
waikiki_last_year = session.query(Measurement).\
filter(Measurement.station.in_(subquery)).\
filter(Measurement.date.between(startdate,lastdate))

In [ ]:
tobs = []
for row in waikiki_last_year:
    tobs.append(row.tobs)

In [ ]:
fig, ax = plt.subplots(tight_layout=True, sharey=True)

ax.hist(tobs, bins=12, label='tobs')

ax.set_xlabel('temperature')
ax.set_ylabel('no. of instances')
ax.set_title('no. of instances of temperature at Waikiki')

ax.legend()

plt.savefig('results/waikiki-histogram.png')

plt.show()

# Close session

In [ ]:
# Close Session
session.close()